<a href="https://colab.research.google.com/github/eternallysunny/ELTE_AI-ML_Final_Project/blob/main/Data_prep_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
#!pip install labelme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 6.8 MB/s 
     |████████████████████████████████| 7.7 MB 22.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 8.3 MB 48.7 MB/s 
     |████████████████████████████████| 59.9 MB 93 kB/s 
     |████████████████████████████████| 338 kB 56.2 MB/s 
  Created wheel for labelme: filename=labelme-5.0.1-py3-none-any.whl size=1466204 sha256=7f6744a3d568563d5054b61d50b38c21c2dd97e2a984b4a78eb65712ae23433e
  Stored in directory: /root/.cache/pip/wheels/e4/f1/e1/84b4d6e95299dbc58c1616c63622624e39643ee591866cfd1e
  Created wheel for imgviz: filename=imgviz-1.5.0-py3-none-any.whl size=7680458 sha256=ecbf93e776b356f85a3902464c5713e3cdd3f106b5db37b1c5c44273f25996aa
  Stored in directory: /root/.cache/pip/wheels/a1/36/f3/3c810b6e34387a0307

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import pandas as pd
import skimage.io as io
import labelme
import json
import random
import cv2
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Add, Conv2D, MaxPooling2D, AveragePooling2D, UpSampling2D, ZeroPadding2D, Activation, concatenate, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50

### For visualizing the outputs ###
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

# Data loader and visualization function definitions

https://towardsdatascience.com/master-the-coco-dataset-for-semantic-image-segmentation-part-2-of-2-c0d1f593096a

https://towardsdatascience.com/master-the-coco-dataset-for-semantic-image-segmentation-part-1-of-2-732712631047

https://github.com/virafpatrawala/COCO-Semantic-Segmentation/blob/master/COCOdataset_SemanticSegmentation_Demo.ipynb

In [ ]:
def filterDataset(folder, classes=None, mode='train'):    
    # initialize COCO api for instance annotations
    annFile = '{}/annotations/instances_{}2014.json'.format(folder, mode)
    coco = COCO(annFile)
    
    images = []
    if classes!=None:
        # iterate for each individual class in the list
        for className in classes:
            # get all images containing given categories
            catIds = coco.getCatIds(catNms=className)
            imgIds = coco.getImgIds(catIds=catIds)
            images += coco.loadImgs(imgIds)
    
    else:
        imgIds = coco.getImgIds()
        images = coco.loadImgs(imgIds)
    
    # Now, filter out the repeated images
    unique_images = []
    for i in range(len(images)):
        if images[i] not in unique_images:
            unique_images.append(images[i])
            
    random.shuffle(unique_images)
    dataset_size = len(unique_images)
    
    return unique_images, dataset_size, coco

In [ ]:
def getClassName(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return None

def getImage(imageObj, img_folder, input_image_size):
    # Read and normalize an image
    train_img = io.imread(img_folder + '/' + imageObj['file_name'])/255.0
    # Resize
    train_img = cv2.resize(train_img, input_image_size)
    if (len(train_img.shape)==3 and train_img.shape[2]==3): # If it is a RGB 3 channel image
        return train_img
    else: # To handle a black and white image, increase dimensions to 3
        stacked_img = np.stack((train_img,)*3, axis=-1)
        return stacked_img
    
def getNormalMask(imageObj, classes, coco, catIds, input_image_size):
    annIds = coco.getAnnIds(imageObj['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    cats = coco.loadCats(catIds)
    train_mask = np.zeros(input_image_size)
    for a in range(len(anns)):
        className = getClassName(anns[a]['category_id'], cats)
        pixel_value = classes.index(className)+1
        new_mask = cv2.resize(coco.annToMask(anns[a])*pixel_value, input_image_size)
        train_mask = np.maximum(new_mask, train_mask)

    # Add extra dimension for parity with train_img size [X * X * 3]
    train_mask = train_mask.reshape(input_image_size[0], input_image_size[1], 1)
    return train_mask  
    
def getBinaryMask(imageObj, coco, catIds, input_image_size):
    annIds = coco.getAnnIds(imageObj['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    train_mask = np.zeros(input_image_size)
    for a in range(len(anns)):
        new_mask = cv2.resize(coco.annToMask(anns[a]), input_image_size)
        
        #Threshold because resizing may cause extraneous values
        new_mask[new_mask >= 0.5] = 1
        new_mask[new_mask < 0.5] = 0

        train_mask = np.maximum(new_mask, train_mask)

    # Add extra dimension for parity with train_img size [X * X * 3]
    train_mask = train_mask.reshape(input_image_size[0], input_image_size[1], 1)
    return train_mask


def dataGeneratorCoco(images, classes, coco, folder, 
                      input_image_size=(224,224), batch_size=4, mode='train', mask_type='binary'):
    
    img_folder = '{}/images/{}'.format(folder, mode)
    dataset_size = len(images)
    catIds = coco.getCatIds(catNms=classes)
    
    c = 0
    while(True):
        img = np.zeros((batch_size, input_image_size[0], input_image_size[1], 3)).astype('float')
        mask = np.zeros((batch_size, input_image_size[0], input_image_size[1], 1)).astype('float')

        for i in range(c, c+batch_size): #initially from 0 to batch_size, when c = 0
            imageObj = images[i]
            
            ### Retrieve Image ###
            train_img = getImage(imageObj, img_folder, input_image_size)
            
            ### Create Mask ###
            if mask_type=="binary":
                train_mask = getBinaryMask(imageObj, coco, catIds, input_image_size)
            
            elif mask_type=="normal":
                train_mask = getNormalMask(imageObj, classes, coco, catIds, input_image_size)                
            
            # Add to respective batch sized arrays
            img[i-c] = train_img
            mask[i-c] = train_mask
            
        c+=batch_size
        if(c + batch_size >= dataset_size):
            c=0
            random.shuffle(images)
        yield img, mask

In [ ]:
def visualizeGenerator(gen):
    img, mask = next(gen)
    
    fig = plt.figure(figsize=(20, 10))
    outerGrid = gridspec.GridSpec(1, 2, wspace=0.1, hspace=0.1)
    
    for i in range(2):
        innerGrid = gridspec.GridSpecFromSubplotSpec(2, 2,
                        subplot_spec=outerGrid[i], wspace=0.05, hspace=0.05)

        for j in range(4):
            ax = plt.Subplot(fig, innerGrid[j])
            if(i==1):
                ax.imshow(img[j])
            else:
                ax.imshow(mask[j][:,:,0])
                
            ax.axis('off')
            fig.add_subplot(ax)        
    plt.show()

In [ ]:
# Load, mask and visualize test data (functions)
def getTestImages(imageObj, imageFolder, input_image_size):     
    # Read and normalize data
    train_image = io.imread(imageFolder + '/' + imageObj + '.jpg')/255.0
    # Resize images
    train_image = cv2.resize(train_image, input_image_size)
    if (len(train_image.shape)==3 and train_image.shape[2]==3):  # If it is an RGB 3 channel image
        return train_image
    else: # If it is a black and white image - to increase dimensions to 3
        stacked_image = np.stack((train_image,)*3, axis=-1)
        return stacked_image
    
def getTestBinaryMask(imageObj, annotFolder, input_image_size):
    train_mask = np.zeros(input_image_size)
    # Load masks from .json files, created with labelme API independently
    with open(annotFolder + '/' + imageObj + '.json', "r", encoding="utf-8") as f:
        dj = json.load(f)
        mask = labelme.utils.shape_to_mask((dj['imageHeight'],
                                                dj['imageWidth']),
                                                dj['shapes'][0]['points'], 
                                                shape_type=None, 
                                                line_width=1, 
                                                point_size=1)
    mask_img = mask.astype('float32')
 
    new_mask = cv2.resize(mask_img, input_image_size)
    # Threshold because resizing may cause extraneous values
    new_mask[new_mask >= 0.5] = 1
    new_mask[new_mask < 0.5] = 0
        
    train_mask = np.maximum(new_mask, train_mask)
    # Add extra dimensions for parity with train_image size (h * w * 3)
    train_mask = train_mask.reshape(input_image_size[0], input_image_size[1], 1)
    return train_mask

def dataGeneratorTest(imgList, imageFolder, annotFolder, input_image_size):

    dataset_size = len(imgList)
    
    img = np.zeros((dataset_size, input_image_size[0], input_image_size[1], 3)).astype('float')
    mask = np.zeros((dataset_size, input_image_size[0], input_image_size[1], 1)).astype('float')
    
    for i in range(0, dataset_size):
        imageObj = imgList[i]
        
        train_img = getTestImages(imageObj, imageFolder, input_image_size)
        
        train_mask = getTestBinaryMask(imageObj, annotFolder, input_image_size)

        img[i] = train_img
        mask[i] = train_mask
        
    return img, mask 

def visualizeTestData(images, masks):
    fig = plt.figure(figsize = (35, 10))
    outerGrid = gridspec.GridSpec(1, 2, wspace = 0.1, hspace = 0.1)
    
    for i in range(2):
        innerGrid = gridspec.GridSpecFromSubplotSpec(2, 4, subplot_spec = outerGrid[i], 
                                                     wspace = 0.025, hspace = 0.025)
        for j in range(8):
            ax = plt.Subplot(fig, innerGrid[j])
            if (i==1):
                ax.imshow(images[j])
            else:
                ax.imshow(masks[j][:, :, 0])
                
            ax.axis('off')
            fig.add_subplot(ax)
    plt.show()

In [ ]:
def visualizeTestPredictions(images, masks, pred):
    fig = plt.figure(figsize = (35, 10))
    outerGrid = gridspec.GridSpec(1, 3, wspace = 0.1, hspace = 0.1)
    
    for i in range(3):
        innerGrid = gridspec.GridSpecFromSubplotSpec(2, 4, subplot_spec = outerGrid[i], 
                                                     wspace = 0.025, hspace = 0.025)
        for j in range(8):
            ax = plt.Subplot(fig, innerGrid[j])
            if (i==0):
                ax.imshow(images[j])
            elif (i==1):
                ax.imshow(masks[j][:, :, 0])
            else:
                ax.imshow(pred[j][:, :, 0])
                
            ax.axis('off')
            fig.add_subplot(ax)
    plt.show()